# 0.0 IMPORTS

## 01. Libraries

In [1]:
import warnings
import inflection
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import dask.dataframe as dd

from dask.distributed import Client, progress

In [2]:
# import datetime
# import numpy as np
# import scipy.stats  as ss

# from IPython.display         import Image
# from IPython.core.display    import HTML

# import matplotlib.pyplot as plt
# import seaborn as sns

# 0.2 Notebook Config

In [3]:
warnings.filterwarnings("ignore")

In [4]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:53744 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.98 GB


In [5]:
# def jupyter_settings():
#     %matplotlib inline
    
#     plt.style.use('bmh')
#     plt.rcParams['figure.figsize'] = [25, 12]
#     plt.rcParams['font.size'] = 24
    
#     display( HTML('<style>.container { width:100% !important; }</style>'))
    
#     warnings.filterwarnings("ignore")
    
#     sns.set()

In [6]:
# seed = 42
# np.random.seed(seed)

# jupyter_settings()

## 0.3 Helper Functions

In [7]:
def rename_columns(df):
    '''Transforms the names of the columns of a dataframe into the snakecase format
    
    Params
    ------
    df: Dataframe to have the columns converted
    
    Return
    ------
    Returns the renamed columns
    
    '''
    # function to convert into snakecase format
    snakecase = lambda col: inflection.underscore(col)
    # mapping the function to convert into snakecase format with the dataframe columns
    new_columns = list(map(snakecase, df.columns))
    # rename the colums
    df.columns = new_columns
    return df.columns

## 0.4 Load Data

### 0.4.1 Load Raw Data

#### 0.4.1.1 train

In [8]:
types = {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,
         'Ruta_SAK':np.uint16, 'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,
         'Venta_uni_hoy':np.uint16,'Venta_hoy':np.float64,'Dev_uni_proxima':np.uint16,
         'Dev_proxima':np.float64, 'Demanda_uni_equil':np.float16}

data_train_raw = dd.read_csv('../00-Data/csv_data/train.csv', usecols=types.keys(), dtype=types)

#### 0.4.1.2 producto_tabla

In [9]:
types = {'Producto_ID':np.uint16,
         'NombreProducto':'object'}


data_producto_raw = dd.read_csv('../00-Data/csv_data/producto_tabla.csv', usecols=types.keys(), dtype=types)

#### 0.4.1.3 cliente_tabla

In [10]:
types = {'Cliente_ID':np.uint32,
         'NombreCliente':'object'}


data_client_raw = dd.read_csv('../00-Data/csv_data/cliente_tabla.csv', usecols=types.keys(), dtype=types)

#### 0.4.1.4 town_state

In [11]:
types = {'Agencia_ID':np.uint16,
         'Town':'object',
         'State':'object'}


data_town_state_raw = dd.read_csv('../00-Data/csv_data/town_state.csv', usecols=types.keys(), dtype=types)

In [12]:
# data_train_raw = dd.read_csv('../00-Data/csv_data/train.csv')
# data_producto_raw = dd.read_csv('../00-Data/csv_data/producto_tabla.csv')
# data_cliente_raw = dd.read_csv('../00-Data/csv_data/cliente_tabla.csv')
# data_town_state_raw = dd.read_csv('../00-Data/csv_data/town_state.csv')

### 0.4.2 Merge Dataset

In [13]:
# data_raw = dd.merge(data_train_raw, data_producto_raw, how='left', on='Producto_ID')
# data_raw = dd.merge(data_raw, data_cliente_raw, how='left', on='Cliente_ID')
# data_raw = dd.merge(data_raw, data_town_state_raw, how='left', on='Agencia_ID')
# data_raw = data_raw.drop(['Producto_ID', 'Cliente_ID', 'Agencia_ID'], axis=1)
# data_raw.head()

# 1.0 DATA DESCRIPTION

## 1.1 Training

In [14]:
df_train_01 = data_train_raw.copy()

### 1.1.1 Columns

In [15]:
df_train_01.columns

Index(['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID',
       'Producto_ID', 'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima',
       'Dev_proxima', 'Demanda_uni_equil'],
      dtype='object')

### 1.1.2 Rename Columns

In [16]:
rename_columns(df_train_01)

Index(['semana', 'agencia_id', 'canal_id', 'ruta_sak', 'cliente_id',
       'producto_id', 'venta_uni_hoy', 'venta_hoy', 'dev_uni_proxima',
       'dev_proxima', 'demanda_uni_equil'],
      dtype='object')

### 1.1.3 Data Dimensions

In [17]:
print(f'Number of Rows: {df_train_01.shape[0].compute()}')
print(f'Number of Columns: {df_train_01.shape[1]}')

Number of Rows: 74180464
Number of Columns: 11


### 1.1.4 Data Types

In [18]:
df_train_01.dtypes

semana                 uint8
agencia_id            uint16
canal_id               uint8
ruta_sak              uint16
cliente_id            uint32
producto_id           uint16
venta_uni_hoy         uint16
venta_hoy            float64
dev_uni_proxima       uint16
dev_proxima          float64
demanda_uni_equil    float16
dtype: object

### 1.1.5 Missing Values

In [19]:
%%time
df_train_01.isnull().sum().compute()

Wall time: 11.6 s


semana               0
agencia_id           0
canal_id             0
ruta_sak             0
cliente_id           0
producto_id          0
venta_uni_hoy        0
venta_hoy            0
dev_uni_proxima      0
dev_proxima          0
demanda_uni_equil    0
dtype: int64

### 1.1.6 ~~Fillout NA~~

There is no NA values

## 1.6 Descriptive Statistical

In [20]:
num_attributes = df_train_01.select_dtypes(exclude=['object'])

In [21]:
describe = num_attributes.describe().compute().T

describe['Relative Std'] = describe['std'] / describe['mean']
describe['range'] = describe['max'] - describe['min']
describe['IQR'] = describe['75%'] - describe['25%']
describe = describe[['min', 'max', 'range', 'mean','std', '50%', 'IQR', '25%', '75%', 'Relative Std']]
describe.columns = ['Min', 'Max', 'Range', 'Mean', 'Std', 'Median', 'IQR', '25%', '75%', 'Relative Std']
describe

,Min,Max,Range,Mean,Std,Median,IQR,25%,75%,Relative Std
semana,3.0,9.000000e+00,6.000000e+00,5.950021e+00,2.013175e+00,6.0,4.00,4.00,8.00,0.338348
agencia_id,1110.0,2.575900e+04,2.464900e+04,2.536509e+03,4.075124e+03,1622.0,720.00,1334.00,2054.00,1.606588
canal_id,1.0,1.100000e+01,1.000000e+01,1.383181e+00,1.463266e+00,1.0,0.00,1.00,1.00,1.057899
ruta_sak,1.0,9.991000e+03,9.990000e+03,2.114855e+03,1.487744e+03,1612.0,1690.00,1213.00,2903.00,0.703473
cliente_id,26.0,2.015152e+09,2.015152e+09,1.802119e+06,2.349577e+06,1464178.0,1989265.00,482299.00,2471564.00,1.303786
producto_id,41.0,4.999700e+04,4.995600e+04,2.084081e+04,1.866392e+04,35113.0,40965.00,2233.00,43198.00,0.895547
venta_uni_hoy,0.0,7.200000e+03,7.200000e+03,7.310163e+00,2.196734e+01,4.0,6.00,2.00,8.00,3.005040
venta_hoy,0.0,6.473600e+05,6.473600e+05,6.854452e+01,3.389795e+02,34.5,45.31,18.86,64.17,4.945392
dev_uni_proxima,0.0,5.339200e+04,5.339200e+04,1.276073e-01,7.465950e+00,0.0,0.00,0.00,0.00,58.507253
dev_proxima,0.0,1.307600e+05,1.307600e+05,1.243248e+00,3.921552e+01,0.0,0.00,0.00,0.00,31.542798


## 1.2 Product

In [22]:
df_producto_01 = data_producto_raw.copy()

### 1.2.1 Columns

In [23]:
df_producto_01.columns

Index(['Producto_ID', 'NombreProducto'], dtype='object')

### 1.2.2 Rename Columns

In [24]:
rename_columns(df_producto_01)

Index(['producto_id', 'nombre_producto'], dtype='object')

### 1.2.3 Data Dimensions

In [25]:
print(f'Number of Rows: {df_producto_01.shape[0].compute()}')
print(f'Number of Columns: {df_producto_01.shape[1]}')

Number of Rows: 2592
Number of Columns: 2


### 1.2.4 Data Types

In [26]:
df_producto_01.dtypes

producto_id        uint16
nombre_producto    object
dtype: object

### 1.2.5 Missing Values

In [27]:
df_producto_01.isnull().sum().compute()

producto_id        0
nombre_producto    0
dtype: int64

### 1.2.6 ~~Fillout NA~~

There is no NA values

## 1.3 Client

In [28]:
df_client_01 = data_client_raw.copy()

### 1.3.1 Columns

In [29]:
df_client_01.columns

Index(['Cliente_ID', 'NombreCliente'], dtype='object')

### 1.3.2 Rename Columns

In [30]:
rename_columns(df_client_01)

Index(['cliente_id', 'nombre_cliente'], dtype='object')

### 1.3.3 Data Dimensions

In [31]:
print(f'Number of Rows: {df_client_01.shape[0].compute()}')
print(f'Number of Columns: {df_client_01.shape[1]}')

Number of Rows: 935362
Number of Columns: 2


### 1.3.4 Data Types

In [32]:
df_client_01.dtypes

cliente_id        uint32
nombre_cliente    object
dtype: object

### 1.3.5 Missing Values

In [33]:
df_client_01.isnull().sum().compute()

cliente_id        0
nombre_cliente    0
dtype: int64

### 1.3.6 ~~Fillout NA~~

There is no NA values

## 1.4 Town State

In [34]:
df_town_state_01 = data_town_state_raw.copy()

### 1.4.1 Columns

In [35]:
df_town_state_01.columns

Index(['Agencia_ID', 'Town', 'State'], dtype='object')

### 1.4.2 Rename Columns

In [36]:
rename_columns(df_town_state_01)

Index(['agencia_id', 'town', 'state'], dtype='object')

### 1.4.3 Data Dimensions

In [37]:
print(f'Number of Rows: {df_town_state_01.shape[0].compute()}')
print(f'Number of Columns: {df_town_state_01.shape[1]}')

Number of Rows: 790
Number of Columns: 3


### 1.4.4 Data Types

In [38]:
df_town_state_01.dtypes

agencia_id    uint16
town          object
state         object
dtype: object

### 1.4.5 Missing Values

In [39]:
df_town_state_01.isnull().sum().compute()

agencia_id    0
town          0
state         0
dtype: int64

### 1.4.6 ~~Fillout NA~~

There is no NA values

# 2.0 FEATURE ENGINEERING

In [40]:
df_train_02 = df_train_01.copy()
df_producto_02 = df_producto_01.copy()
df_client_02 = df_client_01.copy()
df_town_state_02 = df_town_state_01.copy()

## 2.1 Product

In [41]:
df_producto_02 = df_producto_02.drop_duplicates(subset=['producto_id'])

In [42]:
## Weigh
#### Regex weight
df_producto_02['weight'] = df_producto_02['nombre_producto'].str.extract(r'(\d+)\s?(kg|Kg|KG|g|G)')[0]
#### Regex unit of measure: weight
df_producto_02['weight_unit'] = df_producto_02['nombre_producto'].str.extract(r'(\d+)\s?(kg|Kg|KG|g|G)')[1]
#### List of index that the weight need to be correct
weight_kg_id = list(df_producto_02['producto_id'][(~df_producto_02['weight'].isnull()) & (df_producto_02['weight'].str.startswith('0'))].compute())
#### Convert the weight to float
df_producto_02['weight'] = df_producto_02['weight'].astype('float')
#### DataFrame with all the records that don't need to be correct
df_weight_01 = df_producto_02[~df_producto_02['producto_id'].isin(weight_kg_id)]
#### DataFrame with all the records that need to be correct
df_weight_02 = df_producto_02[df_producto_02['producto_id'].isin(weight_kg_id)]
#### Weight correct
df_weight_02['weight'] = df_weight_02['weight']/10
#### Concatenate both dataframes into one
df_producto_02 = dd.concat([df_weight_01, df_weight_02], axis=0)
#### delete both dataframe that was used
del df_weight_01, df_weight_02

#### dict to mapping the unit of measure
weight_mapping = {
    'kg': 1000,
    'Kg': 1000,
    'KG': 1000,
    'g': 1,
    'G': 1
}

#### Adjust to the same scale (kg and g)
df_producto_02['weight_g'] = df_producto_02['weight'] * df_producto_02['weight_unit'].map(weight_mapping)

## Volume
#### Regex volume
df_producto_02['volume'] = df_producto_02['nombre_producto'].str.extract(r'(\s?\d+\s?(l|L|ml|mL|ML)\s)')[0]
#### Getting only the number of volume
volume = df_producto_02['volume'].str.extract(r'(\d+)')
#### Convert to float and rename
volume = volume.astype('float').rename(columns={0:'volume_ml'})
#### Concatenate both dataframes into one
df_producto_02 = dd.concat([df_producto_02, volume], axis=1)
#### delete dataframe that was used
del volume

## Pieces
#### regex pieces (e.g.: 6p)
df_producto_02['pieces_p'] =  df_producto_02['nombre_producto'].str.extract('(\d+)p', expand=False).astype('float')
df_producto_02['pieces_p'] = df_producto_02['pieces_p'].fillna(.0)
#### regex pieces (e.g.: 6P)
df_producto_02['pieces_P'] =  df_producto_02['nombre_producto'].str.extract('(\d+)P', expand=False).astype('float')
df_producto_02['pieces_P'] = df_producto_02['pieces_P'].fillna(.0)
#### regex pieces (e.g.: 3X10)
df_producto_02['pieces_X'] =  df_producto_02['nombre_producto'].str.extract('(\d+)X', expand=False).astype('float')
df_producto_02['pieces_X'] = df_producto_02['pieces_X'].fillna(.0)
#### Sum the possibles pieces
df_producto_02['pieces'] = df_producto_02['pieces_p'] + df_producto_02['pieces_P'] + df_producto_02['pieces_X']
#### replace 0.0 to 1.0
df_producto_02['pieces'] = df_producto_02['pieces'].map(lambda row: 1.0 if row == 0.0 else row)

#### weight / pieces
df_producto_02['weith_pieces'] = df_producto_02['weight_g'] / df_producto_02['pieces']
df_producto_02['weith_pieces'] = df_producto_02['weith_pieces'].fillna(.0)
#### volume / pieces
df_producto_02['volume_pieces'] = df_producto_02['volume_ml'] / df_producto_02['pieces']
df_producto_02['volume_pieces'] = df_producto_02['volume_pieces'].fillna(.0)
#### weight or volume/ pieces
df_producto_02['weight_volume_pieces'] = df_producto_02['weith_pieces'] + df_producto_02['volume_pieces']


#### regex slices (e.g.: 28Reb)
df_producto_02['slices'] =  df_producto_02['nombre_producto'].str.extract('(\d+)Reb', expand=False).astype('float')

#### regex Brands (e.g.: CU LON)
brand = df_producto_02['nombre_producto'].str.extract('.\w\s(\D+) \d+$').rename(columns={0:'brand'})
#### Concatenate both dataframes into one
df_producto_02 = dd.concat([df_producto_02, brand], axis=1)
#### Fill the Products that was not possible to determine the brand
df_producto_02['brand'] = df_producto_02['brand'].fillna('NO_BRAND')
#### delete dataframe that was used
del brand

## Product characteristics
#### New column with the product name in lowercase format
df_producto_02['nombre_producto_lower'] = df_producto_02['nombre_producto'].map(lambda row: row.lower())
#### Chocolate
df_producto_02['flavor_choc'] = df_producto_02['nombre_producto_lower'].str.contains('choc') * 1
#### Vanilla or Cream
df_producto_02['flavor_vainill'] = df_producto_02['nombre_producto_lower'].str.contains('vainill|crem') * 1
#### cinnamon
df_producto_02['flavor_canel'] = df_producto_02['nombre_producto_lower'].str.contains('canel') * 1
#### Integral
df_producto_02['flavor_integral'] = df_producto_02['nombre_producto_lower'].str.contains('integ|cereal|fibra') * 1
#### Fruit
df_producto_02['flavor_fruits'] = df_producto_02['nombre_producto_lower'].str.contains('frut') * 1
#### Frozen storage
df_producto_02['frozen_storage'] = df_producto_02['nombre_producto_lower'].str.contains('congelado') * 1
#### Premium Product
df_producto_02['premium_product'] = df_producto_02['nombre_producto_lower'].str.contains('premium') * 1

#### DataFrame Columns Filtering
df_producto_02 = df_producto_02[['producto_id', 'weight_g', 'volume_ml', 'pieces', 'weight_volume_pieces', 'slices', 'brand', 'flavor_choc',
                                'flavor_vainill', 'flavor_canel', 'flavor_integral', 'flavor_fruits', 'frozen_storage', 'premium_product']]

In [43]:
#df_producto_02['nombre_producto_lower'].sample(0.02).compute()

## 2.2 Client

In [44]:
df_client_02 = df_client_02.drop_duplicates(subset=['cliente_id'])

In [45]:
df_client_02['nombre_cliente'] = df_client_02['nombre_cliente'].map(lambda row: row.lower())

In [46]:
df_client_02['client_type'] = df_client_02['nombre_cliente']

In [47]:
## Special Groups and large companys
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*remision.*','Consignment')
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*wal mart.*|.*sams club.*','Walmart', regex=True)
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*oxxo.*','Oxxo')
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*conasupo.*','Govt')
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*bimbo.*','Bimbo')

### School
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*coleg.*|.*univ.*|.*escu.*|.*insti.*|.*prepar.*','SCHOOL', regex=True)
### Gas 
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*puesto.*','POST')
### Hospitals ans pharmacies
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*farm.*|.*hospital.*|.*clini.*','HOSPITALS', regex=True)
### Eatery
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*cafe.*|.*cremaria.*|.*dulceria.*\
                                                            |.*rest.*|.*burg.*|.*taco.*|.*torta.*|.*taquer.*\
                                                            |.*hot dog.*|.*comedor.*|.*eria.*|.*snack.*','EATERY', regex=True)
### Markets
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*comercial.*|.*bodega.*|.*deposito.*\
                                                            |.*abarrotes.*|.*mercado.*|.*cambio.*|.*market.*|.*mart.*\
                                                            |.*plaza.*|.*misc.*|.*eleven.*|.*exp.*|.*papelaria.*\
                                                            |.*carniceria.*|.*local.*|.*comodin.*|.*providencia.*','MARKET', regex=True)
### Fresh Markets
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*verdu.*|.*frut.*','FRESH MARKET', regex=True)
### Hotel
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*hotel.*|.*motel.*','HOTEL', regex=True)
### Small Franchise
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*la .*|.*el .*|.*de .*|.*del .*|.*san.*\
                                                            |.*los .*|.*y .*|.*las .*|.*mi .*|.*ag .*|.*ma .*\
                                                            |.*ii .*|.*[0-9]+.*','SMALL FRANCHISE', regex=True)

### No identificado
### Hotel
df_client_02['client_type'] = df_client_02['client_type'].str.replace('.*no identificado.*|.*sin nombre.*','Unidentified', regex=True)

df_client_02['client_type'] = df_client_02['client_type'].map(lambda row: 'individual' if row.islower() else row)

In [48]:
teste = df_client_02['client_type'].value_counts().compute()

In [49]:
teste.head(50)

individual         377324
Unidentified       282032
SMALL FRANCHISE    149852
MARKET              63440
EATERY              30950
Oxxo                 9276
HOSPITALS            5723
SCHOOL               5562
POST                 2658
HOTEL                1107
FRESH MARKET         1068
Govt                  958
Bimbo                 319
Walmart               217
Consignment            14
Name: client_type, dtype: int64

## 2.3 Town State

In [52]:
df_town_state_02 = df_town_state_02.drop_duplicates(subset=['agencia_id'])

In [54]:
df_town_state_02.head()

,agencia_id,town,state
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."


In [73]:
teste = df_town_state_02['town'].value_counts()

In [74]:
teste.compute()

2013 AG. MEGA NAUCALPAN    8
2355 JALAPA I              7
2011 AG. SAN ANTONIO       7
2161 IRAPUATO GUADALUPE    6
2396 RUIZ CORTINEZ         6
                          ..
2389 EMILIANO ZAPATA       1
2404 VALLADOLID            1
2055 Pinotepa              1
2410 CANCUN II             1
3216 NUEVO LAREDO          1
Name: town, Length: 260, dtype: int64

In [62]:
df_town_state_02[df_town_state_02['town'].str.contains('AG. MEGA')].compute()

,agencia_id,town,state
8,1119,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
20,1139,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
22,1142,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
55,1221,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
60,1228,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
508,3221,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
547,20139,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
618,22013,2013 AG. MEGA NAUCALPAN,ESTADO DE MÉXICO
